In [1]:
import pandas as pd
import os
from tqdm import tqdm

# Reduce decimal points to 2
pd.options.display.float_format = '{:,.2f}'.format

# initialise progress_apply
tqdm.pandas(desc="my bar!")

In [2]:
# Import date and drop useless column
path = os.getcwd()

data = pd.read_csv('{}/data_assignment_1.csv'.format(path))

data.drop('Unnamed: 0', axis=1, inplace=True)

In [3]:
# take a first look at the number of NAs
print(data.isna().sum())
initial_length = len(data)

id              0
year            0
yearest     67916
industry    79437
pertot      79437
enggrad     73439
sales       79541
va          79835
gom         79843
rdint       79720
reext       79544
ipnc        81662
ipnf        81665
ipnm        81660
ipr         79454
patent      79504
dtype: int64


In [4]:
print('Si on supprimait directement tous les NAs, il nous resterait', len(data.dropna()), 'lignes', '\n')
print('Attention, le dataframe ressemblerait à un gruyère (plusieurs années disparaîtraient pour chaque entreprise)')

Si on supprimait directement tous les NAs, il nous resterait 30742 lignes 

Attention, le dataframe ressemblerait à un gruyère (plusieurs années disparaîtraient pour chaque entreprise)


In [5]:
# loop to delete companies with more than 11 NAs in a column
print('deleting companies with more than 11 NAs in a column', '\n')
for i in tqdm(data['id'].unique()):

    for column in data.columns:
        # print(data.loc[data['id'] == i, [column]].isna().sum().iloc[0])
        if data.loc[data['id'] == i, [column]].isna().sum().iloc[0] > 11:
            data.drop(data[data['id'] == i].index, inplace=True)
            # print('deleting company', i)
            break


# loop to fill the NAs for each company
print('filling NAs in the columns of the remaining companies', '\n')
for i in tqdm(data['id'].unique()):
        
    try: 
        # fill NAs with the variable mode for industry and yearest (makes more sense to our minds)
        data.loc[data['id'] == i, ['industry']] = data.loc[data['id'] == i, ['industry']].mode(axis=0, dropna=True).iloc[0, 0]
        data.loc[data['id'] == i, ['yearest']] = data.loc[data['id'] == i, ['yearest']].mode(axis=0, dropna=True).iloc[0, 0]

        # interpolate the columns where variables are continuous
        continuous_variables = ['enggrad', 'sales', 'va', 'gom','rdint', 'reext']
        data.loc[data['id'] == i, continuous_variables] = data.loc[data['id'] == i, continuous_variables].interpolate(method='linear', axis=0, limit_direction='both')

        # interpolate the columns where variables are discrete
        discrete_variables = ['pertot', 'patent']
        data.loc[data['id'] == i, discrete_variables] = data.loc[data['id'] == i, discrete_variables].interpolate(method='linear', axis=0, limit_direction='both').round(0)

        # fill NAs in binary variables
        binary_variables = ['ipnc', 'ipnf', 'ipnm', 'ipr']
        data.loc[data['id'] == i, binary_variables] = data.loc[data['id'] == i, binary_variables].interpolate(method='linear', axis=0, limit_direction='both')

    except IndexError:
        pass

deleting companies with more than 11 NAs in a column 



100%|██████████| 5304/5304 [01:13<00:00, 72.20it/s] 


filling NAs in the columns of the remaining companies 



100%|██████████| 1216/1216 [00:17<00:00, 68.45it/s]


In [6]:
len_after_cleaning = len(data)
print('number of observations:', len_after_cleaning, '\n')
print(initial_length - len_after_cleaning, 'rows have been deleted during the process', '\n')
print('number of NAs for each column:', '\n', data.isna().sum())

number of observations: 27968 

94024 rows have been deleted during the process 

number of NAs for each column: 
 id          0
year        0
yearest     0
industry    0
pertot      0
enggrad     0
sales       0
va          0
gom         0
rdint       0
reext       0
ipnc        0
ipnf        0
ipnm        0
ipr         0
patent      0
dtype: int64


Attention, il reste moins de lignes en faisant ce savage cleaning que lorsque l'on supprime tous les NAs "sans réfléchir". 

In [7]:
# Create age variable
data.insert(loc=3, column='age', value=data.apply(lambda row: row['year'] - row['yearest'], axis=1), allow_duplicates=True)

In [8]:
# Compute the sales growth rate
data['sales_growth_rate'] = data.sort_values('year').groupby('id')['sales'].pct_change(fill_method=None)

# Compute the 90th percentile of the sales growth rate for each year
percentile_90_thresholds = data.groupby('year')['sales_growth_rate'].quantile(0.9)

# Create a binary target variable based on the 90th percentile
data['is_hgf'] = data.progress_apply(lambda row: 1 if row['sales_growth_rate'] >= percentile_90_thresholds[row['year']] else 0, axis=1)

C:\Users\EMCCd\AppData\Local\Temp\ipykernel_27432\3459315293.py:2: FutureWarning: The 'fill_method' and 'limit' keywords in SeriesGroupBy.pct_change are deprecated and will be removed in a future version. Call ffill before calling pct_change instead.
  data['sales_growth_rate'] = data.sort_values('year').groupby('id')['sales'].pct_change(fill_method=None)
my bar!:   0%|          | 0/27968 [00:00<?, ?it/s]

my bar!: 100%|██████████| 27968/27968 [00:00<00:00, 73143.77it/s]


In [9]:
# drop the remaining NAs (in the sales_growth_rate and is_hgf variables) to perform model training and predictions
data = data.dropna()
print(len_after_cleaning - len(data), 'more rows have been deleted')

1216 more rows have been deleted


In [10]:
# change data types

data_types_dict = {'id': int,
                   'year': int,
                   'yearest': int,
                   'age': int,
                   'industry': int,
                   'pertot': int,
                   'enggrad': float,
                   'sales': float,
                   'va': float,
                   'gom': float,
                   'rdint': float,
                   'reext': float,
                   'ipnc': bool,
                   'ipnf': bool,
                   'ipnm': bool,
                   'ipr': bool,
                   'patent': int,
                   'sales_growth_rate': float,
                   'is_hgf': bool}

data = data.astype(data_types_dict)

In [11]:
# checking for missing values
data.isnull().sum()

id                   0
year                 0
yearest              0
age                  0
industry             0
pertot               0
enggrad              0
sales                0
va                   0
gom                  0
rdint                0
reext                0
ipnc                 0
ipnf                 0
ipnm                 0
ipr                  0
patent               0
sales_growth_rate    0
is_hgf               0
dtype: int64

In [12]:
#Change the True_False values of the is_hgf column to 1 and 0
data['is_hgf'] = data['is_hgf'].astype(int)

In [13]:
# checking the distribution of Target Varibale
data['is_hgf'].value_counts()

is_hgf
0    23610
1     3142
Name: count, dtype: int64

In [14]:
# get the mean of the Target variable for each columns  
data.groupby('is_hgf').mean()

,id,year,yearest,age,industry,pertot,enggrad,sales,va,gom,rdint,reext,ipnc,ipnf,ipnm,ipr,patent,sales_growth_rate
is_hgf,,,,,,,,,,,,,,,,,,
0,"1,523.42","2,001.35","1,972.95",28.41,10.19,273.56,4.75,"68,136,701.74","26,815,388.45",7.18,"502,902.83","391,148.23",0.13,0.12,0.13,0.33,0.36,-0.00
1,"1,534.76","2,002.59","1,974.81",27.78,10.92,256.56,5.34,"71,969,726.84","40,097,080.69",9.26,"610,071.89","453,694.89",0.13,0.12,0.12,0.35,0.35,1.64


In [15]:
# import the packages
import numpy as np
import pandas as pd
import sklearn.datasets
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [16]:
# Determine the in-put and out-put variables
X = data.drop(columns='is_hgf', axis=1)
Y = data['is_hgf']

In [17]:
# Split the data into training and test data 
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state=0)


In [18]:
# Print
print(X.shape, X_train.shape, X_test.shape)

(26752, 18) (20064, 18) (6688, 18)


In [19]:
# Model training 
#Logistic Regression 
model = LogisticRegression()

# training the Logistic Regression model using Training data
model.fit(X_train, Y_train)


LogisticRegression()

In [20]:
#Model Evaluation :
# accuracy on training data
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(Y_train, X_train_prediction)  

In [21]:
print('Accuracy on training data = ', training_data_accuracy)

Accuracy on training data =  0.883622408293461


In [22]:
# accuracy on test data
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(Y_test, X_test_prediction)

In [23]:
print('Accuracy on test data = ', test_data_accuracy)

Accuracy on test data =  0.8793361244019139


In [24]:
# We can change the value of C and play with the flexibility of the model
logreg001 = LogisticRegression(max_iter=5000, C=0.001).fit(X_train, Y_train)
logreg50 = LogisticRegression(max_iter=5000, C=50).fit(X_train, Y_train)

print("Accuracy C=0.001 (test): {:.3f}".format(logreg001.score(X_test, Y_test)))
print("Accuracy C=50 (test): {:.3f}".format(logreg50.score(X_test, Y_test)))

Accuracy C=0.001 (test): 0.879
Accuracy C=50 (test): 0.879


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge, RidgeCV, Lasso, LassoCV
from sklearn.metrics import mean_squared_error

In [ ]:
X = data.drop(['sales_growth_rate'], axis = 1)
y = data['sales_growth_rate']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)


In [ ]:
# Instantiate the model
ridge = Ridge(normalize = True)

# Fit the model on the training data
ridge = ridge.fit(X_train, y_train)

# Visualize coefficients
print(pd.Series(ridge.coef_, index = X.columns))

# Check the performance of the model
print('MSE (training): %.2f' % mean_squared_error(y_train, ridge.predict(X_train)),
      'MSE (test): %.2f' % mean_squared_error(y_test, ridge.predict(X_test)), sep='\n')


In [ ]:
# Set manually some values for alpha
alphas = 10**np.linspace(5,-2,100)*0.5
alphas

coefs = []
for a in alphas:
    ridge = Ridge(alpha=a, fit_intercept=False)
    ridge.fit(X_train, y_train)
    coefs.append(ridge.coef_)

#Plot ridge coefficients as a function of the regularization
ax = plt.gca()
ax.plot(alphas, coefs)
ax.set_xscale('log')
plt.xlabel('alpha')
plt.ylabel('coefficient')
plt.title('Ridge coefficients profile')
plt.axis('tight')
plt.show()

# We use cross-validation to choose the tuning parameter
ridgecv = RidgeCV(alphas = alphas, cv = 10, scoring = 'neg_mean_squared_error', normalize = True)
ridgecv.fit(X_train, y_train)
ridgecv.alpha_


print('MSE (training): %.2f' % mean_squared_error(y_train, ridgecv.predict(X_train)),
      'MSE (test): %.2f' % mean_squared_error(y_test, ridgecv.predict(X_test)), sep='\n')


In [ ]:
lassocv = LassoCV(alphas = None, cv = 10, max_iter = 100000, normalize = True)
lassocv.fit(X_train, y_train)
lassocv.alpha_

# How many coeffiecient are set to zero?
print('Number of features used:', np.sum(lassocv.coef_ != 0))

print('MSE (training): %.2f' % mean_squared_error(y_train, lassocv.predict(X_train)),
      'MSE (test): %.2f' % mean_squared_error(y_test, lassocv.predict(X_test)), sep='\n')

